In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
from fastai.vision.all import *

In [2]:
# Go to project root folder
%cd ..

/home/shubham/Desktop/sign-language-mnist


In [3]:
# Read config
import pprint

with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

pprint.pprint(config)

{'base': {'project': 'sign language mnist classification', 'random_seed': 42},
 'data_load': {'target': 'label',
               'test_data_path': 'data/sign_mnist_test/sign_mnist_test.csv',
               'train_data_path': 'data/sign_mnist_train/sign_mnist_train.csv'},
 'model': {'architecture': 'resnet34',
           'pretrained': True,
           'save_path': 'models/model.pkl'},
 'train': {'fine_tune_args': {'base_lr': 0.001, 'epochs': 8},
           'loss_func': 'CrossEntropyLoss',
           'lr': 0.001,
           'metrics': ['accuracy', 'f1_score'],
           'num_epochs': 10,
           'optimizer': 'Adam',
           'weight_decay': 0.01}}


In [5]:
train_df = pd.read_csv(config['data_load']['train_data_path'])
test_df = pd.read_csv(config['data_load']['test_data_path'])

In [6]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [7]:
### Function that converts all columns starting with pixel into img and returns others column and pixels replaced with img
def get_images(df): 
    IMG_WIDTH = 28
    IMG_HEIGHT = 28
    
    df['img'] = df[df.columns[df.columns.str.startswith('pixel')]].apply(
        lambda x : PILImage(Image.fromarray(np.array(x.values).reshape((IMG_WIDTH, IMG_HEIGHT)).astype(np.uint8))),axis=1)
    
    return df[df.columns[[not x for x in df.columns.str.startswith('pixel')]]]

In [8]:
train_df = get_images(train_df)
test_df = get_images(test_df)

In [9]:
train_df.head()

,label,img
0,3,PILImage mode=L size=28x28
1,6,PILImage mode=L size=28x28
2,2,PILImage mode=L size=28x28
3,2,PILImage mode=L size=28x28
4,13,PILImage mode=L size=28x28


In [10]:
def get_x(r): return r['img']
def get_y(r): return r['label']
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), get_x = get_x, get_y=get_y)
dls = dblock.dataloaders(train_df)

In [11]:
learn = vision_learner(dls, resnet34, metrics=error_rate).to_fp16()
learn.fine_tune(20)

/home/shubham/Desktop/sign-language-mnist/dvc-venv/lib/python3.8/site-packages/fastai/vision/learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/home/shubham/Desktop/sign-language-mnist/dvc-venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shubham/Desktop/sign-language-mnist/dvc-venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
dl = learn.dls.test_dl(list(test_df['img']))
inp,preds,_,dec_preds = learn.get_preds(dl=dl, with_input=True, with_decoded=True)
test_df['Label'] = dec_preds

In [ ]:
learn.export(fname=(Path("models") / "model.pkl").absolute())